# Простой пример модели детектирования акустических событий.

## Импортируем необходимые библиотеки

In [ ]:
import numpy as np
from librosa.feature import melspectrogram
from librosa.core import load as load_wav
from librosa.display import specshow
from librosa import power_to_db
import os
import matplotlib.pyplot as plt

from utils.preprocessing import extract_log_mel_feats

## Скачивание данных и подготовка к обучению

Скачать архив с данными можно, перейдя по ссылке. Для скачивания потребуется авторизация на kaggle.com. 

Затем необходимо положить данные для трейна и теста в папки data/audio_train и data/audio_test соответственно, так же в корень папки необходимо положить файл train.csv, который также можно найти в архиве. 

Далее задаются различные параметры для извлечения фичей из файлов.

In [ ]:
train_folder = './data/audio_train'
test_folder = './data/audio_train'


sample_rate = 8000
fft_size = 1024
overlap = 4
hop = fft_size // overlap
mels = 64

## Визуализация фичей, используемых при обучении, для одного файла.

In [ ]:
wav_data, sr = load_wav(os.path.join(train_folder, 'aaf6167e.wav'), sr=sample_rate)

mel_spec = melspectrogram(wav_data, n_fft=fft_size, hop_length=hop, n_mels=mels, 
                          fmax=sample_rate//2)

plt.figure(figsize=(10, 4))
specshow(mel_spec, y_axis='mel',
         fmax=sample_rate//2, x_axis='time')
plt.colorbar(format='%+2.0f дБ')
plt.title('Мелспектрограмма скрипки')
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
specshow(power_to_db(mel_spec, ref=np.max), y_axis='mel',
         fmax=sample_rate//2, x_axis='time')
plt.colorbar(format='%+2.0f дБ')
plt.title('Логарифм мелспектрограммы скрипки')
plt.tight_layout()
plt.show()

## Извлечение признаков


Для ускорения вычислений нам будет полезно вычислить всем мелспектрограммы заранее и сохранить их. Выполнение данной ячейки займет некоторое время, так как процесс вычисления признаков дорогой. Размер полученных файлов будет приблизительно 1 гб каждый (при исходных значениях параметров).

In [ ]:
path_to_csv = os.path.join('./data', 'train.csv')
pickle_train_path = os.path.join('./data', 'train.pickle')
pickle_test_path = os.path.join('./data', 'test.pickle')

extract_log_mel_feats('train', path_to_csv, train_folder, pickle_train_path, sample_rate, fft_size, hop, mels)
extract_log_mel_feats('test', '', test_folder, pickle_test_path, sample_rate, fft_size, hop, mels)

## Обучаем модель

In [1]:
import pickle


from model.batcher import Batcher
from model.config import Config
from model.cnn_model import CNNModel
from model.utils import prepare_shape

from keras.callbacks import ModelCheckpoint

In [ ]:
config = Config(shape=(64, 63, 1), learning_rate=0.001, n_classes=41)

pickle_train_data = pickle.load(open(pickle_train_path, 'rb'))

x_val = []
y_val = []

x_tr = []
y_tr = []

for i, row in enumerate(pickle_train_data):
    feature = prepare_shape(row['feature'], config)
    if i < 1000:
        x_val.append(feature)
        y_val.append(row['label_id'])
    else:
        x_tr.append(feature)
        y_tr.append(row['label_id'])
x_val = np.asarray(x_val)
x_tr = np.asarray(x_tr)

y_val = np.asarray(y_val)
y_tr = np.asarray(y_tr)

train_batcher = Batcher(x_tr, y_tr, batch_size=100)
val_batcher = Batcher(x_val, y_val)

checkpoint = ModelCheckpoint('./best_weights.h5', monitor='val_loss',
                             verbose=1, save_best_only=True)

model = CNNModel(config).get_model()

history = model.fit_generator(train_batcher,
                              callbacks=[checkpoint],
                              validation_data=val_batcher,
                              epochs=50,
                              use_multiprocessing=False,
                              workers=0)

## Формируем файл для отправки на kaggle.

In [ ]:
pickle_test_data = pickle.load(open('./test.pickle', 'rb'))
x_test = []

for i, row in enumerate(pickle_test_data):
    feature = prepare_shape(row['feature'], config)
    x_test.append(feature)
x_test = np.asarray(x_test)
model.load_weights('./best_weights.h5')
config = Config(shape=(64, 63, 1), learning_rate=0.001, n_classes=41)

pred_batcher = Batcher(x_test)

predictions = model.predict_generator(pred_batcher,
                                      use_multiprocessing=False,
                                      workers=0)
res = np.argsort(predictions)
id_to_label = {i: label for label, i in label_to_id.items()}
with open('./base_submission.csv', 'w') as f:
    f.write('fname,label\n')
    for i, row in enumerate(res):
        row = row[::-1]
        f.write('{},{} {} {}\n'.format(pickle_test_data[i]['fname'],
                                       id_to_label[row[0]],
                                       id_to_label[row[1]],
                                       id_to_label[row[2]]))

### Готово! 

Теперь в рабочей директории появился файл с предсказаниями для тестовой выборки - его можно смело отправлять на kaggle и, например, попробовать улучшить модель и сравнить результат.